Table of contents

1. Create bare-bones minerva model

2. Replicate original paper finding (requires better interface)

3. Create rhythm/tone experiment with fixed
    - rhythm/tone split
    - number participants
    - participant stimuli

4. Make experiment configurable

5. Run a bazillion experiments

# Bare-bones Minerva2 model

In [69]:
import numpy as np

In [70]:
len([val for val in np.unique(np.array((1,0,-1,1, 1, 1, 1, 0))) if val not in (-1, 0, 1)])

0

## The Model (v0.1)

In [71]:
class Minerva2:
    def __init__(self, features_per_trace):
        self.features_per_trace = features_per_trace
        self.model = []
        
    def get_activation(self, probe, trace):
        '''
        also returns similarity in 2nd return value
        '''
        # the @ symbol denotes a "dot product," which is the same as 
        # similarity = sum([probe[i] * self.model[trace_idx][i] for i in range(len(probe))])
        similarity = (probe @ trace) / len(probe)
        return similarity**3, similarity
        
    def get_activation_by_idx(self, probe, trace_idx):
        return self.get_activation(probe, self.model[trace_idx], return_sim)
    
    def get_echo_intensity(self, probe, return_all=False):
        activations = []
        similarities = []
        for trace in self.model:
            activation, similarity = self.get_activation(probe, trace)
            activations.append(activation)
            similarities.append(similarity)
        
        if return_all:
            return sum(activations) / len(activations), activations, similarities
        else:
            return sum(activations) / len(activations)
    
    def add_trace(self, trace):
        if type(trace) != np.ndarray:
            raise Exception("Trace is not of type numpy array, fail.")
        if trace.shape != (self.features_per_trace,):
            raise Exception("Trace is not a one-dimensional array of length", self.features_per_trace, ", fail.")
        if len([x for x in trace if x not in (-1, 0, 1)]) > 0:
            raise Exception("Trace contains values besides -1, 0, or 1, fail.")
        self.model.append(trace)

    def pretty_print(self, probe):
        '''
        Illustrates the calculation of echo intensity
        '''
        total_activation, activations, similarities = self.get_echo_intensity(probe, return_all=True)

        print('PROBE:', list(probe))
        for i, (activation, similarity) in enumerate(zip(activations, similarities)):
            print('TRACE {}:'.format(i), list(self.model[i]), '->', '{:>6.0f}^3 = {:>8.0f}'.format(similarity, activation))
        print('-'*80)
        print('{:>80.0f}'.format(total_activation))

## Try it out with some ZERO traces

In [72]:
model1_fpt = 10

In [73]:
model1 = Minerva2(model1_fpt)

In [74]:
for _ in range(10):
    rand_trace = np.zeros(model1_fpt)
    model1.add_trace(rand_trace)

In [75]:
model1.model

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])]

Let's create a random probe to test

In [76]:
probe1 = np.random.randint(-1, 2, model1_fpt)
probe1

array([ 1,  1,  0, -1,  1,  1,  0, -1, -1,  1])

In [77]:
model1.get_echo_intensity(probe1, return_all=True)

(0.0,
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [78]:
model1.pretty_print(probe1)

PROBE: [1, 1, 0, -1, 1, 1, 0, -1, -1, 1]
TRACE 0: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] ->      0^3 =        0
TRACE 1: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] ->      0^3 =        0
TRACE 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] ->      0^3 =        0
TRACE 3: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] ->      0^3 =        0
TRACE 4: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] ->      0^3 =        0
TRACE 5: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] ->      0^3 =        0
TRACE 6: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] ->      0^3 =        0
TRACE 7: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] ->      0^3 =        0
TRACE 8: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] ->      0^3 =        0
TRACE 9: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] ->      0^3 =        0
--------------------------------------------------------------------------------
                                                

Blank memory means a blank response! Looking good so far.

## Now test with random traces

In [79]:
model2_fpt = 8

In [80]:
model2 = Minerva2(model2_fpt)

In [81]:
for _ in range(12):
    rand_trace = np.random.randint(-1, 2, model2_fpt)
    model2.add_trace(rand_trace)

In [82]:
model2.model

[array([ 0,  0, -1,  1,  0, -1,  0, -1]),
 array([-1,  0,  0,  0,  1, -1, -1, -1]),
 array([-1,  0,  0,  0,  0,  0,  0,  0]),
 array([-1,  0,  1,  1,  1,  1,  1, -1]),
 array([ 0,  0,  0,  0,  0,  1, -1, -1]),
 array([ 1,  0, -1,  1, -1,  1,  1,  0]),
 array([ 1, -1,  0,  0, -1, -1,  0,  1]),
 array([0, 1, 0, 1, 0, 0, 0, 0]),
 array([ 1, -1,  1, -1, -1, -1,  1, -1]),
 array([-1, -1, -1,  0, -1, -1, -1,  0]),
 array([ 1, -1, -1, -1,  1,  1, -1,  0]),
 array([ 0, -1, -1,  1, -1,  0,  1,  0])]

And a random test probe...

In [83]:
probe2 = np.random.randint(-1, 2, model2_fpt)
probe2

array([ 1, -1,  1,  1,  0, -1, -1,  1])

In [84]:
model2.get_echo_intensity(probe2, return_all=True)

(0.008951822916666666,
 [0.0,
  0.0,
  -0.001953125,
  -0.015625,
  -0.001953125,
  -0.001953125,
  0.125,
  0.0,
  0.001953125,
  0.001953125,
  0.0,
  0.0],
 [0.0, 0.0, -0.125, -0.25, -0.125, -0.125, 0.5, 0.0, 0.125, 0.125, 0.0, 0.0])

In [85]:
model2.pretty_print(probe2)

PROBE: [1, -1, 1, 1, 0, -1, -1, 1]
TRACE 0: [0, 0, -1, 1, 0, -1, 0, -1] ->      0^3 =        0
TRACE 1: [-1, 0, 0, 0, 1, -1, -1, -1] ->      0^3 =        0
TRACE 2: [-1, 0, 0, 0, 0, 0, 0, 0] ->     -0^3 =       -0
TRACE 3: [-1, 0, 1, 1, 1, 1, 1, -1] ->     -0^3 =       -0
TRACE 4: [0, 0, 0, 0, 0, 1, -1, -1] ->     -0^3 =       -0
TRACE 5: [1, 0, -1, 1, -1, 1, 1, 0] ->     -0^3 =       -0
TRACE 6: [1, -1, 0, 0, -1, -1, 0, 1] ->      0^3 =        0
TRACE 7: [0, 1, 0, 1, 0, 0, 0, 0] ->      0^3 =        0
TRACE 8: [1, -1, 1, -1, -1, -1, 1, -1] ->      0^3 =        0
TRACE 9: [-1, -1, -1, 0, -1, -1, -1, 0] ->      0^3 =        0
TRACE 10: [1, -1, -1, -1, 1, 1, -1, 0] ->      0^3 =        0
TRACE 11: [0, -1, -1, 1, -1, 0, 1, 0] ->      0^3 =        0
--------------------------------------------------------------------------------
                                                                               0
